In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import pywt
import matplotlib.pyplot as plt

from mne import read_epochs, set_log_level, compute_rank, concatenate_epochs

from scipy.stats import kurtosis, skew, moment, entropy, norm
from mne.decoding import CSP, Scaler
from mrmr import mrmr_classif
from random import randint
from ReliefF import ReliefF
from sklearn import svm
from sklearn.decomposition import PCA
from sklearn.feature_selection import RFE
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from pyriemann.tangentspace import TangentSpace
from pyriemann.estimation import Covariances, Kernels
from sklearn.model_selection import train_test_split, LeaveOneOut, StratifiedShuffleSplit, StratifiedKFold, cross_val_score, GridSearchCV,ShuffleSplit
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

from jupyterthemes.stylefx import set_nb_theme
set_nb_theme('gruvboxd')

In [2]:
set_log_level('warning')
epochs = read_epochs('ica_epo.fif').pick('eeg').filter(0,240)
epochs.drop_channels(epochs.info['bads'])
epochs.apply_baseline((-1.4,-0.4))

Number of events,105
Events,left: 22r_pinch: 20r_stop: 22rest: 17right: 24
Time range,-2.000 – 7.999 sec
Baseline,-1.400 – -0.400 sec


In [3]:
lda = LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto')
nuSvm = svm.NuSVC(gamma="auto")
knn = KNeighborsClassifier(n_neighbors=3)
relief = ReliefF(n_neighbors=3, n_features_to_keep=10)
linear_svm = svm.SVR(kernel="linear")
svm_rbf = svm.SVC(kernel="rbf")
rf = RandomForestClassifier(random_state=3, max_depth=25)
le = LabelEncoder()
tangent_space = TangentSpace()
scaler = Scaler(info=epochs.info)

In [4]:
def dwt_det_coeff(x, db='db2'):
    aprx, det = pywt.dwt(x,db)
    return det

def dwt_aprox_coeff(x, db='db2'):
    aprx, det = pywt.dwt(x,db)
    return aprx

def rms(x):
    return np.sqrt(np.mean(x**2))

def slope(x):
    t = np.linspace(0, len(x)-1, len(x))
    return np.polyfit(t, x, 1)[0]

def autocorr(x):
    return float(np.correlate(x,x))
    
def temp_centroid(x):
    nom = [x[i] * i for i in range(x.shape[0])]
    return sum(nom)/sum(x)

def energy(x):
    return sum(x**2)

def med_abs_diff(x):
    return np.median(np.abs(np.diff(x)))

def mean_abs_diff(x):
    return np.mean(np.abs(np.diff(x)))

def calc_centroid(x, fs=1024):
    energy = np.array(x) ** 2
    t = range(len(x))
    t = [float(x) / fs for x in t]
    t_energy = np.dot(np.array(t), np.array(energy))
    energy_sum = np.sum(energy)

    if energy_sum == 0 or t_energy == 0:
        centroid = 0
    else:
        centroid = t_energy / energy_sum

    return centroid


left vs rest

In [5]:
conditions = ['left','rest']
subset = epochs[conditions].copy()
subset.drop_channels(subset.info['bads'])
subset = subset.pick(['eeg'])
subset = subset.apply_baseline((-1.4,-0.4))
y = le.fit_transform(subset.events[:,2])
train_data = subset.copy().crop(0.4,1.2).get_data()    
time_config = (3,0.4,300,100)

In [6]:
y = le.fit_transform(subset.events[:,2])
chance = np.mean(y == y[0])
chance = max(chance, 1. - chance)

In [28]:
print(genConf(train_data.shape[0], 0.01))
print(genConf(train_data.shape[0], 0.05))

75.53856358406144
70.81913317142232


In [16]:
test_dwt('db14', 4)

0.7307692307692306   0.85  pca
0.7461538461538461   0.87  pca
0.7692307692307692   0.89  pca


In [8]:
pca = PCA(n_components=.89)

In [16]:
test_extra_lvl('db14', 4, 6, 'lda')

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:06<00:00,  1.50it/s]


pca 0.6692307692307693
rfe 0.8
mrmr 0.8923076923076921
rf 0.5384615384615385


In [12]:
classifiers = ['svm', 'lda', 'rf', 'knn']
selectors = ['rfe', 'mrmr', 'relief', 'pca']

scores = []
for selec in selectors:
    score = test_lvl_set('db4', 3, 4, 'lda', selec)
    scores.append(score)

print('rfe', scores[0], 'mrmr',  scores[1], 'relief', scores[2], 'pca', scores[3])
np.save('dwt_selectors_left_rest.npy', np.array(scores))

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:07<00:00,  1.33it/s]


rfe 0.7692307692307692 mrmr 0.8384615384615385 relief 0.40769230769230774 pca 0.6538461538461539


In [13]:
scores = []
for classifier in classifiers:
    score = test_lvl_set('db4', 3, 4, classifier, 'mrmr')
    scores.append(score)

print('svm', scores[0], 'lda',  scores[1], 'rf', scores[2], 'knn', scores[3])
np.save('dwt_classifiers_left_rest.npy', np.array(scores))

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:02<00:00,  3.52it/s]


svm 0.7 lda 0.8384615384615385 rf 0.8076923076923077 knn 0.6615384615384615


informative features

In [13]:
selected_fts = get_informative_features('db4', 3, 4, 'lda')
selected_fts = np.concatenate(selected_fts)
np.save('left_rest_fix_features', selected_fts)

100%|███████████████████████████████████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.75it/s]


mean accuracy:  0.8461538461538461


In [14]:
selected_fts_custom = get_informative_features('db14', 3, 4, 'lda')
selected_fts_custom = np.concatenate(selected_fts_custom)
np.save('left_rest_custom_features', selected_fts)

100%|███████████████████████████████████████████████████████████████████████████████| 10/10 [00:03<00:00,  3.23it/s]


mean accuracy:  0.9153846153846154


left vs right

In [5]:
conditions = ['left','right']
subset = epochs[conditions].copy()
subset.drop_channels(subset.info['bads'])
subset = subset.pick(['eeg'])
subset = subset.apply_baseline((-1.4,-0.4))
y = le.fit_transform(subset.events[:,2])
train_data = subset.copy().crop(0.0,8).get_data()    
time_config = (3,0.4,300,100)

In [16]:
y = le.fit_transform(subset.events[:,2])
chance = np.mean(y == y[0])
chance = max(chance, 1. - chance)

In [33]:
print(genConf(train_data.shape[0], 0.01))
print(genConf(train_data.shape[0], 0.05))

70.38055881374699
65.98502897053197


In [18]:
test_dwt('db2', 6)

0.6387499999999999   0.85  pca


In [20]:
test_levels('db2')

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:11<00:00,  1.10s/it]


0.8266666666666668 with  1 and  2


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:09<00:00,  1.02it/s]


0.8400000000000001 with  1 and  5


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:06<00:00,  1.65it/s]


0.8533333333333333 with  1 and  6


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:10<00:00,  1.03s/it]


0.8866666666666667 with  2 and  6


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:06<00:00,  1.58it/s]


0.9400000000000001 with  5 and  6


In [34]:
test_extra_lvl('db2', 2, 6, 'lda')

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.96it/s]


pca 0.42333333333333334
rfe 0.7899999999999999
mrmr 0.8733333333333334
rf 0.41833333333333333


In [16]:
scores = []
for selec in selectors:
    score = test_lvl_set('db4', 3, 4, 'lda', selec)
    scores.append(score)
print('rfe', scores[0], 'mrmr',  scores[1], 'pca' , scores[3],'relief', scores[2] )
np.save('dwt_selectors_left_right.npy', np.array(scores))

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:02<00:00,  3.78it/s]


rfe 0.6833333333333333 mrmr 0.8666666666666668 pca 0.42791666666666667 relief 0.44375


In [17]:
scores = []
for classifier in classifiers:
    score = test_lvl_set('db4', 3, 4, classifier, 'mrmr')
    scores.append(score)
print('svm', scores[0], 'lda',  scores[1], 'rf', scores[2], 'knn', scores[3])
np.save('dwt_classifiers_left_right.npy', np.array(scores))


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:02<00:00,  4.12it/s]


svm 0.5995833333333332 lda 0.8666666666666668 rf 0.7612500000000001 knn 0.5716666666666667


informative features

In [9]:
selected_fts = get_informative_features('db4', 3, 4, 'lda')
selected_fts = np.concatenate(selected_fts)
np.save('left_right_fix_features', selected_fts)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:06<00:00,  1.62it/s]


mean accuracy:  0.8 std 0.10749676997731401


In [10]:
selected_fts_custom = get_informative_features('db4', 1, 4, 'lda')
selected_fts_custom = np.concatenate(selected_fts_custom)
np.save('left_right_custom_features_1', selected_fts_custom)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:06<00:00,  1.60it/s]


mean accuracy:  0.7295833333333334 std 0.0955330440214275


4 levels


In [7]:
selected_fts_custom = get_informative_features('db4', 5, 6, 4, 15)

100%|██████████████████████████████████████████████████████████████████████████████████| 15/15 [00:05<00:00,  2.69it/s]


mean accuracy:  0.8666666666666666 +- 0.0843274042711568


In [17]:
np.save('4_lvl_features', selected_fts_custom)
np.save('4_lvl_score', score)

3 class

In [9]:
conditions = ['left','right','rest']
subset = epochs[conditions].copy()
subset.drop_channels(subset.info['bads'])
subset = subset.pick(['eeg'])
subset = subset.apply_baseline((-1.4,-0.4))
y = le.fit_transform(subset.events[:,2])
train_data = subset.copy().crop(0.2,1).get_data()    
time_config = (3,0.4,300,100)
y = le.fit_transform(subset.events[:,2])
chance = np.max([np.mean(y == y[0]), np.mean(y == y[1]), np.mean(y == y[2])])
print(genConf(train_data.shape[0], 0.01))
print(genConf(train_data.shape[0], 0.05))

54.26824946419225
50.57388605239645


In [11]:
test_levels('db2')

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:07<00:00,  1.33it/s]


0.6666666666666667 with  1 and  2


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:11<00:00,  1.18s/it]


0.6857142857142857 with  1 and  4


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:06<00:00,  1.49it/s]


0.7047619047619047 with  1 and  5


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:10<00:00,  1.05s/it]


0.7333333333333334 with  1 and  6


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.96it/s]


KeyboardInterrupt: 

In [7]:
def p(n):
    return (n*chance+2)/(n+4)
def conf(n,p,alpha):
    return np.sqrt((p*(1-p))/(n+4))*norm.ppf(1-(alpha/2))
def genConf(n, alpha):
    res = np.zeros((n))
    for i in range(0,n):
        res[i] = p(i) + conf(i, p(i), alpha)
    return res[-1]*100

In [6]:
def get_feature_names(lvl, lvl1, fourth=False):
    all_features = []
    signal_features = ['std', 'max', 'min', 'rms', 'slope', 'skew', 'avg', 
                       'mean_abs','temp_centr','enrgy','krts','st_centr']
    rg = 4 if fourth else 3
    
    for level in range(rg):
        for ft in signal_features:
            for ch in subset.info['chs']:
                if level == 0:
                    all_features.append('D'+str(lvl) + '_' + ch['ch_name'] + '_' + ft)
                elif level == 1:
                    all_features.append('D'+str(lvl1) + '_' + ch['ch_name'] + '_' + ft)
                elif level == 3:
                    all_features.append('D'+str(fourth) + '_' + ch['ch_name'] + '_' + ft)
                else :
                    all_features.append('A0'+ '_' + ch['ch_name'] + '_' + ft)
                    
    return all_features 

def get_coeffs(db, lvl, lvl1): 
    x_aprox_coeff = train_data
          
    detail_coeffs = []
    aprox_coeffs = []
    for dwt_lvl in range(lvl1+1):
        x_det_coeff = np.apply_along_axis(dwt_det_coeff, 2, x_aprox_coeff, db=db)
        x_aprox_coeff = np.apply_along_axis(dwt_aprox_coeff, 2, x_aprox_coeff, db=db)

        detail_coeffs.append(scaler.fit_transform(x_det_coeff.copy(),y))
        aprox_coeffs.append(scaler.fit_transform(x_aprox_coeff.copy(),y))
    
    return detail_coeffs, aprox_coeffs

def get_informative_features(db, lvl, lvl1, fourth=False, features=10):
    score = []
    ft_names = get_feature_names(lvl, lvl1, fourth) 
    selected_features = []
    detail_coeffs, aprox_coeffs = get_coeffs(db, lvl, lvl1) 
    for train_rep in range(10):  
        cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=randint(20,50))        
        cv_split = cv.split(detail_coeffs[lvl], y)
        
        mrmr_features = None  
        median_score = []
        for train_idx, test_idx in cv_split:
            y_train, y_test = y[train_idx], y[test_idx]  

            x_train, x_test = get_feature_vector(detail_coeffs, 
                                                 aprox_coeffs, 
                                                 lvl, lvl1, 
                                                 train_idx, 
                                                 test_idx, fourth)

            estimator = lda

            if mrmr_features is None:            
                x_pd = pd.DataFrame(x_train)
                mrmr_features = mrmr_classif(X=x_pd, y=y_train, K=features)
                
            x_train =  x_train[:,mrmr_features]
            x_test = x_test[:,mrmr_features]

            estimator.fit(x_train, y_train)
            median_score.append(np.median(estimator.score(x_test, y_test)))
            
        selected_features.append(np.array(ft_names)[mrmr_features])
        score.append(np.median(median_score))
        
    print('mean accuracy: ',np.mean(score), '+-', np.std(score))
    return selected_features, np.mean(score)

def get_feature_vector(detail_coeffs, aprox_coeffs, lvl, lvl1, train_idx, test_idx, fourth=False):
    x_train = []
    
    x_train.append(np.apply_along_axis(np.std, 2, detail_coeffs[lvl][train_idx]))
    x_train.append(np.apply_along_axis(np.max, 2, detail_coeffs[lvl][train_idx]))
    x_train.append(np.apply_along_axis(np.min, 2, detail_coeffs[lvl][train_idx]))
    x_train.append(np.apply_along_axis(rms, 2, detail_coeffs[lvl][train_idx]))
    x_train.append(np.apply_along_axis(slope, 2, detail_coeffs[lvl][train_idx]))
    x_train.append(np.apply_along_axis(skew, 2, detail_coeffs[lvl][train_idx]))
    x_train.append(np.apply_along_axis(np.average, 2, detail_coeffs[lvl][train_idx]**2))
    x_train.append(np.apply_along_axis(mean_abs_diff, 2, detail_coeffs[lvl][train_idx]))
    x_train.append(np.apply_along_axis(temp_centroid, 2, detail_coeffs[lvl][train_idx]))
    x_train.append(np.apply_along_axis(energy, 2, detail_coeffs[lvl][train_idx]))
    x_train.append(np.apply_along_axis(kurtosis, 2, detail_coeffs[lvl][train_idx]))
    x_train.append(np.apply_along_axis(calc_centroid, 2, detail_coeffs[lvl][train_idx]))

    x_train.append(np.apply_along_axis(np.std, 2, detail_coeffs[lvl1][train_idx]))
    x_train.append(np.apply_along_axis(np.max, 2, detail_coeffs[lvl1][train_idx]))
    x_train.append(np.apply_along_axis(np.min, 2, detail_coeffs[lvl1][train_idx]))
    x_train.append(np.apply_along_axis(rms, 2, detail_coeffs[lvl1][train_idx]))
    x_train.append(np.apply_along_axis(slope, 2, detail_coeffs[lvl1][train_idx]))
    x_train.append(np.apply_along_axis(skew, 2, detail_coeffs[lvl1][train_idx]))
    x_train.append(np.apply_along_axis(np.average, 2, detail_coeffs[lvl1][train_idx]**2))
    x_train.append(np.apply_along_axis(mean_abs_diff, 2, detail_coeffs[lvl1][train_idx]))
    x_train.append(np.apply_along_axis(temp_centroid, 2, detail_coeffs[lvl1][train_idx]))
    x_train.append(np.apply_along_axis(energy, 2, detail_coeffs[lvl1][train_idx]))
    x_train.append(np.apply_along_axis(kurtosis, 2, detail_coeffs[lvl1][train_idx]))
    x_train.append(np.apply_along_axis(calc_centroid, 2, detail_coeffs[lvl1][train_idx]))

    x_train.append(np.apply_along_axis(np.std, 2, aprox_coeffs[-1][train_idx]))
    x_train.append(np.apply_along_axis(np.max, 2, aprox_coeffs[-1][train_idx]))
    x_train.append(np.apply_along_axis(np.min, 2, aprox_coeffs[-1][train_idx]))
    x_train.append(np.apply_along_axis(rms, 2, aprox_coeffs[-1][train_idx]))
    x_train.append(np.apply_along_axis(slope, 2, aprox_coeffs[-1][train_idx]))
    x_train.append(np.apply_along_axis(skew, 2, aprox_coeffs[-1][train_idx]))
    x_train.append(np.apply_along_axis(np.average, 2, aprox_coeffs[-1][train_idx]**2))
    x_train.append(np.apply_along_axis(mean_abs_diff, 2, aprox_coeffs[-1][train_idx]))
    x_train.append(np.apply_along_axis(temp_centroid, 2, aprox_coeffs[-1][train_idx]))
    x_train.append(np.apply_along_axis(energy, 2, aprox_coeffs[-1][train_idx]))
    x_train.append(np.apply_along_axis(kurtosis, 2, aprox_coeffs[-1][train_idx]))
    x_train.append(np.apply_along_axis(calc_centroid, 2, aprox_coeffs[-1][train_idx]))
    
    if fourth :
        x_train.append(np.apply_along_axis(np.std, 2, detail_coeffs[fourth][train_idx]))
        x_train.append(np.apply_along_axis(np.max, 2, detail_coeffs[fourth][train_idx]))
        x_train.append(np.apply_along_axis(np.min, 2, detail_coeffs[fourth][train_idx]))
        x_train.append(np.apply_along_axis(rms, 2, detail_coeffs[fourth][train_idx]))
        x_train.append(np.apply_along_axis(slope, 2, detail_coeffs[fourth][train_idx]))
        x_train.append(np.apply_along_axis(skew, 2, detail_coeffs[fourth][train_idx]))
        x_train.append(np.apply_along_axis(np.average, 2, detail_coeffs[fourth][train_idx]**2))
        x_train.append(np.apply_along_axis(mean_abs_diff, 2, detail_coeffs[fourth][train_idx]))
        x_train.append(np.apply_along_axis(temp_centroid, 2, detail_coeffs[fourth][train_idx]))
        x_train.append(np.apply_along_axis(energy, 2, detail_coeffs[fourth][train_idx]))
        x_train.append(np.apply_along_axis(kurtosis, 2, detail_coeffs[fourth][train_idx]))
        x_train.append(np.apply_along_axis(calc_centroid, 2, detail_coeffs[fourth][train_idx]))

    x_train = np.concatenate(x_train, axis=1)

    x_test = []

    x_test.append(np.apply_along_axis(np.std, 2, detail_coeffs[lvl][test_idx]))
    x_test.append(np.apply_along_axis(np.max, 2, detail_coeffs[lvl][test_idx]))
    x_test.append(np.apply_along_axis(np.min, 2, detail_coeffs[lvl][test_idx]))
    x_test.append(np.apply_along_axis(rms, 2, detail_coeffs[lvl][test_idx]))
    x_test.append(np.apply_along_axis(slope, 2, detail_coeffs[lvl][test_idx]))
    x_test.append(np.apply_along_axis(skew, 2, detail_coeffs[lvl][test_idx]))
    x_test.append(np.apply_along_axis(np.average, 2, detail_coeffs[lvl][test_idx]**2))
    x_test.append(np.apply_along_axis(mean_abs_diff, 2, detail_coeffs[lvl][test_idx]))
    x_test.append(np.apply_along_axis(temp_centroid, 2, detail_coeffs[lvl][test_idx]))
    x_test.append(np.apply_along_axis(energy, 2, detail_coeffs[lvl][test_idx]))
    x_test.append(np.apply_along_axis(kurtosis, 2, detail_coeffs[lvl][test_idx]))
    x_test.append(np.apply_along_axis(calc_centroid, 2, detail_coeffs[lvl][test_idx]))

    x_test.append(np.apply_along_axis(np.std, 2, detail_coeffs[lvl1][test_idx]))
    x_test.append(np.apply_along_axis(np.max, 2, detail_coeffs[lvl1][test_idx]))
    x_test.append(np.apply_along_axis(np.min, 2, detail_coeffs[lvl1][test_idx]))
    x_test.append(np.apply_along_axis(rms, 2, detail_coeffs[lvl1][test_idx]))
    x_test.append(np.apply_along_axis(slope, 2, detail_coeffs[lvl1][test_idx]))
    x_test.append(np.apply_along_axis(skew, 2, detail_coeffs[lvl1][test_idx]))
    x_test.append(np.apply_along_axis(np.average, 2, detail_coeffs[lvl1][test_idx]**2))
    x_test.append(np.apply_along_axis(mean_abs_diff, 2, detail_coeffs[lvl1][test_idx]))
    x_test.append(np.apply_along_axis(temp_centroid, 2, detail_coeffs[lvl1][test_idx]))
    x_test.append(np.apply_along_axis(energy, 2, detail_coeffs[lvl1][test_idx]))
    x_test.append(np.apply_along_axis(kurtosis, 2, detail_coeffs[lvl1][test_idx]))
    x_test.append(np.apply_along_axis(calc_centroid, 2, detail_coeffs[lvl1][test_idx]))

    x_test.append(np.apply_along_axis(np.std, 2, aprox_coeffs[-1][test_idx]))
    x_test.append(np.apply_along_axis(np.max, 2, aprox_coeffs[-1][test_idx]))
    x_test.append(np.apply_along_axis(np.min, 2, aprox_coeffs[-1][test_idx]))
    x_test.append(np.apply_along_axis(rms, 2, aprox_coeffs[-1][test_idx]))
    x_test.append(np.apply_along_axis(slope, 2, aprox_coeffs[-1][test_idx]))
    x_test.append(np.apply_along_axis(skew, 2, aprox_coeffs[-1][test_idx]))
    x_test.append(np.apply_along_axis(np.average, 2, aprox_coeffs[-1][test_idx]**2))
    x_test.append(np.apply_along_axis(mean_abs_diff, 2, aprox_coeffs[-1][test_idx]))
    x_test.append(np.apply_along_axis(temp_centroid, 2, aprox_coeffs[-1][test_idx]))
    x_test.append(np.apply_along_axis(energy, 2, aprox_coeffs[-1][test_idx]))
    x_test.append(np.apply_along_axis(kurtosis, 2, aprox_coeffs[-1][test_idx]))
    x_test.append(np.apply_along_axis(calc_centroid, 2, aprox_coeffs[-1][test_idx]))

    if fourth :
        x_test.append(np.apply_along_axis(np.std, 2, detail_coeffs[fourth][test_idx]))
        x_test.append(np.apply_along_axis(np.max, 2, detail_coeffs[fourth][test_idx]))
        x_test.append(np.apply_along_axis(np.min, 2, detail_coeffs[fourth][test_idx]))
        x_test.append(np.apply_along_axis(rms, 2, detail_coeffs[fourth][test_idx]))
        x_test.append(np.apply_along_axis(slope, 2, detail_coeffs[fourth][test_idx]))
        x_test.append(np.apply_along_axis(skew, 2, detail_coeffs[fourth][test_idx]))
        x_test.append(np.apply_along_axis(np.average, 2, detail_coeffs[fourth][test_idx]**2))
        x_test.append(np.apply_along_axis(mean_abs_diff, 2, detail_coeffs[fourth][test_idx]))
        x_test.append(np.apply_along_axis(temp_centroid, 2, detail_coeffs[fourth][test_idx]))
        x_test.append(np.apply_along_axis(energy, 2, detail_coeffs[fourth][test_idx]))
        x_test.append(np.apply_along_axis(kurtosis, 2, detail_coeffs[fourth][test_idx]))
        x_test.append(np.apply_along_axis(calc_centroid, 2, detail_coeffs[fourth][test_idx]))

    x_test = np.concatenate(x_test, axis=1)

    return x_train, x_test
